In [1]:
import glob
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import random
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.preprocessing import LabelEncoder
from keras.applications import resnet50, vgg16
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, Lambda, GlobalAveragePooling2D, BatchNormalization
from keras.models import Model, Sequential
from keras import optimizers
from keras import backend as K
%matplotlib inline

Using TensorFlow backend.


In [2]:
# set to image directory
car = glob.glob('/Users/austinau-yeung/Documents/Georgia Tech/1/ECE6254/project/cardiomegaly2/*')
ede = glob.glob('/Users/austinau-yeung/Documents/Georgia Tech/1/ECE6254/project/edema2/*')

# parameters
# car_train_num = 400
# ede_train_num = 400
car_train_num = 30
ede_train_num = 30
epochs = 5

In [3]:
num_classes = 2

# select random subset of images for training
car_train = np.random.choice(car,size=car_train_num,replace=False)
ede_train = np.random.choice(ede,size=ede_train_num,replace=False)
car = list(set(car)-set(car_train))
ede = list(set(ede)-set(ede_train))
car_test = car
ede_test = ede

car_test_num = len(car_test)
ede_test_num = len(ede_test)

In [4]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_DIM = (IMG_WIDTH,IMG_HEIGHT)

# load training images
car_train_imgs = [img_to_array(load_img(img,target_size=IMG_DIM,color_mode="rgb")) for img in car_train]
ede_train_imgs = [img_to_array(load_img(img,target_size=IMG_DIM,color_mode="rgb")) for img in ede_train]

# create corresponding labels
train_imgs = np.array(car_train_imgs+ede_train_imgs)
train_imgs_scaled = train_imgs.astype('float32')/255
train_labels = car_train_num*['c']+ede_train_num*['e']

# load test images and create corresponding labels
car_test_imgs = [img_to_array(load_img(img,target_size=IMG_DIM,color_mode="rgb")) for img in car_test]
ede_test_imgs = [img_to_array(load_img(img,target_size=IMG_DIM,color_mode="rgb")) for img in ede_test]
test_imgs = np.array(car_test_imgs+ede_test_imgs)
test_imgs_scaled = test_imgs.astype('float32')/255
test_labels = car_test_num*['c']+ede_test_num*['e']

input_shape = (IMG_HEIGHT,IMG_WIDTH,train_imgs.shape[3])

In [5]:
# encode class labels as 0/1
le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
test_labels_enc = le.transform(test_labels)

In [6]:
# siamese network referenced from the following:
# https://github.com/keras-team/keras/blob/master/examples/mnist_siamese.py

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 2
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 1
    return np.mean(pred == y_true)

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 1, y_true.dtype)))
#     return K.mean(y_pred[1])

def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][random.randrange(0,n+1)]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

def vggnet_base(input_shape):

    # train last 2 layers (top 3 layers not included)
    vggnet = vgg16.VGG16(include_top=False,weights='imagenet',input_shape=input_shape,pooling='avg')
    for layer in vggnet.layers[-5:]:
        layer.trainable=True
    for layer in vggnet.layers[:-5]:
        layer.trainable=False
    x = vggnet.output
    
#     imgIn = Input(shape=input_shape)
#     x = Flatten()(imgIn)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     x = BatchNormalization()(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     x = Dense(1, activation='sigmoid')(x)
        
    return Model(vggnet.input,x)
#     return Model(imgIn,x)


In [7]:
# create positive and negative pairs
idx = [np.where(train_labels_enc==i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(train_imgs_scaled,idx)

idx = [np.where(test_labels_enc==i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(test_imgs_scaled,idx)

# setup image augmentation on pairs of images using ImageDataGenerator, referenced from the following:
# https://github.com/keras-team/keras/issues/3386#issuecomment-237555199

def trainGenerator( X, I, Y):

    while True:
        # shuffled indices    
        idx = np.random.permutation( X.shape[0])
        # create image generator
        datagen = ImageDataGenerator(
                fill_mode='constant',
                cval=0,
                rescale=1./1,
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=5, #180,  # randomly rotate images in the range (degrees, 0 to 180)
                width_shift_range=0.05, #0.1,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.05, #0.1,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,  # randomly flip images
                vertical_flip=False)  # randomly flip images

        batches = datagen.flow( X[idx], Y[idx], batch_size=64, shuffle=False)
        idx0 = 0
        for batch in batches:
            idx1 = idx0 + batch[0].shape[0]

            yield [batch[0], I[ idx[ idx0:idx1 ] ]], batch[1]

            idx0 = idx1
            if idx1 >= X.shape[0]:
                break
                
def testGenerator( X, I, Y):

    while True:
        # suffled indices    
        idx = np.random.permutation( X.shape[0])
        # create image generator
        datagen = ImageDataGenerator(
                rescale=1./1)

        batches = datagen.flow( X[idx], Y[idx], batch_size=64, shuffle=False)
        idx0 = 0
        for batch in batches:
            idx1 = idx0 + batch[0].shape[0]

            yield [batch[0], I[ idx[ idx0:idx1 ] ]], batch[1]

            idx0 = idx1
            if idx1 >= X.shape[0]:
                break


In [8]:
# create siamese network with euclidean distance as final layer
base_network = vggnet_base(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 256)          15108672    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]              

In [9]:
adm = optimizers.Adam(lr=0.0001)
model.compile(loss=contrastive_loss, optimizer=adm, metrics=[accuracy])

# use unedited training images
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=100,
#           validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y),
          shuffle=False)

# use augmented training images
# history = model.fit_generator(testGenerator(tr_pairs[:, 0],tr_pairs[:, 1],tr_y), 
#                               steps_per_epoch=1, 
#                               epochs=100,
#                               validation_data=testGenerator(te_pairs[:, 0],te_pairs[:, 1],te_y), 
#                               validation_steps=1, 
#                               verbose=1)

tr_y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, tr_y_pred)
te_y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, te_y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Epoch 1/100
116/116 [==============================] - 45s 387ms/step - loss: 2.8865 - accuracy: 0.5000
Epoch 2/100
116/116 [==============================] - 42s 364ms/step - loss: 1.0247 - accuracy: 0.5603
Epoch 3/100
116/116 [==============================] - 42s 366ms/step - loss: 2.7102 - accuracy: 0.5000
Epoch 4/100
116/116 [==============================] - 43s 367ms/step - loss: 1.6571 - accuracy: 0.5000
Epoch 5/100
116/116 [==============================] - 41s 354ms/step - loss: 1.7744 - accuracy: 0.5000
Epoch 6/100
116/116 [==============================] - 39s 340ms/step - loss: 1.0137 - accuracy: 0.6121
Epoch 7/100
116/116 [==============================] - 40s 342ms/step - loss: 2.2290 - accuracy: 0.5000
Epoch 8/100
116/116 [==============================] - 42s 366ms/step - loss: 1.6096 - accuracy: 0.5000
Epoch 9/100
116/116 [==============================] - 42s 358ms/step - loss: 1.4113 - accuracy: 0.5000
Epoch 10/100
116/116 [==============================] - 42s 366m

116/116 [==============================] - 44s 377ms/step - loss: 0.0676 - accuracy: 1.0000
Epoch 80/100
116/116 [==============================] - 43s 372ms/step - loss: 0.0551 - accuracy: 1.0000
Epoch 81/100
116/116 [==============================] - 42s 363ms/step - loss: 0.0472 - accuracy: 1.0000
Epoch 82/100
116/116 [==============================] - 43s 367ms/step - loss: 0.0519 - accuracy: 1.0000
Epoch 83/100
116/116 [==============================] - 42s 360ms/step - loss: 0.0482 - accuracy: 1.0000
Epoch 84/100
116/116 [==============================] - 43s 372ms/step - loss: 0.0481 - accuracy: 1.0000
Epoch 85/100
116/116 [==============================] - 43s 367ms/step - loss: 0.0369 - accuracy: 1.0000
Epoch 86/100
116/116 [==============================] - 42s 361ms/step - loss: 0.0356 - accuracy: 1.0000
Epoch 87/100
116/116 [==============================] - 43s 370ms/step - loss: 0.0338 - accuracy: 1.0000
Epoch 88/100
116/116 [==============================] - 43s 371ms/st

In [10]:
pred = tr_y_pred.ravel()<1

print(pred) # predicted label by thresholding distance
print(tr_y) # true label
print(pred==tr_y)
print(tr_y_pred) # distance for each pair (alternates between positive and negative)

[ True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False  True False  True False
  True False  True False  True False  True False]
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0]
[ True  True  True  True  True  True  True  True  True  

In [11]:
model.evaluate([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y)

116/116 [==============================] - 36s 312ms/step


[0.0037689048090371593, 1.0]

In [12]:
model.metrics_names

['loss', 'accuracy']

In [13]:
tr_y_pred

array([[0.11923882],
       [1.9698675 ],
       [0.19381183],
       [2.1538377 ],
       [0.00757753],
       [2.1345341 ],
       [0.10463237],
       [1.9717833 ],
       [0.06570432],
       [2.1834223 ],
       [0.08669833],
       [2.192066  ],
       [0.02577663],
       [2.3737206 ],
       [0.05363158],
       [2.154893  ],
       [0.04386137],
       [2.3962922 ],
       [0.06071804],
       [2.0137234 ],
       [0.03964727],
       [2.4096072 ],
       [0.0415711 ],
       [2.0274527 ],
       [0.05896509],
       [2.0224576 ],
       [0.07736355],
       [2.0618322 ],
       [0.02024755],
       [2.234074  ],
       [0.04182813],
       [2.2561235 ],
       [0.04259025],
       [1.9317333 ],
       [0.0154742 ],
       [2.0352252 ],
       [0.00790885],
       [2.0439458 ],
       [0.03879889],
       [2.133123  ],
       [0.04697106],
       [2.0717645 ],
       [0.03018928],
       [2.325931  ],
       [0.00690261],
       [2.0975072 ],
       [0.07625269],
       [2.103